# Introduction

Hi! My name is Tadej, and I would like to present you my small project about the weather.

You were given 2 files in my git repository. You are reading one right now, and the other is a python file. You are probably wondering where is all the data. Worry not, you are to generate it yourself. Just run the "Weather.py" file and type in the command start(). You should be given some new folders. You can type start() anytime you like, when you want to update the data to the current weather forecast. It should work.

The descrpitions will be general, since I can't give examples on cities specifically, as the information will vary day by day.

You should run all the cells bellow now.

# Analysis

In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
data = pd.read_csv('information.csv')

In [ ]:
data.rename(columns={'country': 'Country', 'city': 'City', 'temp': 'Temp(°C)', 'weather': 'Weather',  'wind': 'Wind(km/h)', 'visibility': 'Visibility(km)', 'pressure': 'Pressure(mbar)', 'humidity': 'Humidity(%)', 'dewpoint': 'Dewpoint(°C)', 
                    'rainchance': 'Rainchance(%)', 'rainamount': 'Rainamount(mm)', 'ntemp': 'Temp next day(°C)', 'nwind': 'Wind next day(km/h)', 'nhumidity': 'Humidity next day(%)', 
                    'nrainchance': 'Rainchance next day(%)', 'nrainamount': 'Rainamount next day(mm)'}, inplace=True)

The raw information looks like this:

In [ ]:
data

First off, we should look at some basic analysis.

## Highest temperature

In [ ]:
data[['Country', 'City', 'Temp(°C)']].sort_values(by='Temp(°C)', ascending=False).head(5)

In [ ]:
data.sort_values(by='Temp(°C)', ascending=False).head(20).plot.bar(x='City', y='Temp(°C)')

## Weather, affected by pressure

In [ ]:
data[data['Pressure(mbar)'].isnull() == False].sort_values(by='Pressure(mbar)', ascending=False).tail(30).plot.bar(x='Weather', y='Pressure(mbar)').set_ylim(900, 1050)

In [ ]:
data[data['Pressure(mbar)'].isnull() == False][['Pressure(mbar)', 'Rainchance(%)', 'Rainchance next day(%)', 'Weather']].sort_values(by='Pressure(mbar)').head(10)

In [ ]:
data[data['Pressure(mbar)'].isnull() == False][['Pressure(mbar)', 'Rainchance(%)', 'Rainchance next day(%)', 'Weather']].sort_values(by='Pressure(mbar)').tail(10)

We see, that the chance of rain is dependent on the pressure. There are many other factors, that is why there are some anomalies. The first table represents the lowest pressure in cities, whereas the second table represents the highest.

## What is a dewpoint and relative humidity?

You have probably noticed there is a column, indexed 'dewpoint'. If you are not a meterologist, there is a small chance that you are familiar with this expression. But let's first talk about relative humidity, since the dewpoint is closely related to it.



Relative humidity is, measured in procents, the current amount of water content, relative to the maximum amount of water content in a specific area. It depends on the temperature. Let's say there is a given amount of water in a pocket of air, for instance, 4 grams. Temperature is held at 20°C. The pocket can contain the maximum amount of 15 grams of water, so the relative humidity is somewhere at 27%.

What happens, if we raise the temperature? The air molecules scatter, giving more space, hence the air pocket size increases, giving more space for water. The content of water remains the same, so the relative humidity drops.

What if we lower the temperature? The air pocket gets smaller and can contain less water. When the maximum amount of water the pocket can contain, reaches the actual water content, in our instance, 4 grams, the relative humidity becomes 100%. We say, that the air is fully saturated. We call this temperature the dewpoint. If we bring the temperature even lower, the water will condense to liquid water. The condensed water is called a fog, or a cloud, depending on the altitude it has formed on.

The dew point is called a frostpoint, when it is below 0°C.

The next graph shows us, that the dewpoint is always near the temperature, or lower.

In [ ]:
data.plot.scatter('Temp(°C)', 'Dewpoint(°C)')

The dewpoint and the temperature are closely related to weather, obviously. If the temperature is near the dewpoint, meaning that the air doesn't need to get much cooler to reach the dewpoint, the humidity is high and the weather is usually foggy, cloudy, etc.

In [ ]:
data[data['Temp(°C)'] - data['Dewpoint(°C)'] < 3][['Country', 'City' ,'Weather', 'Humidity(%)']].head(10)

In [ ]:
data[(data['Temp(°C)'] - data['Dewpoint(°C)'] < 3) & (data['Dewpoint(°C)'] < -1)][['Country', 'City' ,'Weather', 'Humidity(%)']].head(10)

When the dewpoint and temperature are the same, humidity is near 100%, and usually there is a high chance of rain.

In [ ]:
data[data['Temp(°C)'] - data['Dewpoint(°C)'] == 0][['Country', 'City' , 'Humidity(%)', 'Rainchance(%)']]

Humidity also affects visibility, most of the dots are in the upper left corner, which is to be expected, as a greater procent of humidity causes weather to be foggy.

In [ ]:
data[data['Visibility(km)'].isnull() == False].plot.scatter(x='Visibility(km)', y='Humidity(%)')

## Differences in conditions

In [ ]:
data['Difference in temperature(°C)'] = data['Temp next day(°C)'] - data['Temp(°C)']

Here we are presented, where the biggest rise in temperature will occur.

In [ ]:
data[['Country', 'City', 'Difference in temperature(°C)', 'Pressure(mbar)']].sort_values(by='Difference in temperature(°C)', ascending=False).head(10)

And the biggest fall:

In [ ]:
data[data['Difference in temperature(°C)'].isnull() == False][['Country', 'City', 'Difference in temperature(°C)', 'Pressure(mbar)']].sort_values(by='Difference in temperature(°C)', ascending=False).tail(10)

## Finding the best place to be in

We will now define the function, which will determine, where it should be the best place for you.

In [ ]:
def best_place(warmer, colder, windy, rain, weather, i_like_to_see_things):
    dictionary = {}
    for city in range(0, len(data)):
        points = 0
        if data.iloc[city]['Temp(°C)'] == data.iloc[city]['Temp(°C)']:
            if data.iloc[city]['Temp(°C)'] >= 0:
                points+=(warmer*data.iloc[city]['Temp(°C)']) // 3
            else:
                points+=(colder*abs(data.iloc[city]['Temp(°C)'])) // 3
        if weather in data.iloc[city]['Weather']:
            points+=20
        if (data.iloc[city]['Wind(km/h)'] == data.iloc[city]['Wind(km/h)']):
            if (data.iloc[city]['Wind(km/h)'] != 'No wind'):
                points+=(windy-3)*int(data.iloc[city]['Wind(km/h)'])
        if data.iloc[city]['Visibility(km)'] == data.iloc[city]['Visibility(km)']:
            if i_like_to_see_things == True:
                points+=data.iloc[city]['Visibility(km)']//5
            else:
                points-=data.iloc[city]['Visibility(km)']//5
                         
        if data.iloc[city]['Rainamount(mm)'] == data.iloc[city]['Rainamount(mm)']:
            points+=(data.iloc[city]['Rainamount(mm)'] * rain) // 3
        dictionary[data.iloc[city]['City']] = points
    max_value = max(dictionary.values())
    max_keys = [k for k, v in dictionary.items() if v == max_value]
    return max_keys
    
    

Inputs are the following:


warmer = How do you like the warmth, from 0 to 5

colder = How do you like the cold, from 0 to 5

windy = How do you like the wind, from 0 to 5

rainy = How do you like the rain, form 0 to 5

weather = Which weather do you like the most (Sun, Rain, Fog, Cloud, Haze, Clear) - it is important to capitalize the first letter

i_like_to_see_things = self explanatory, values are True or False

In [ ]:
best_place(5, 0, 0, 1, 'Rain', True)

# Conclusion

Thank you for reviewing my project! I hope you learned something.